# IE 306  - Assignment 1 - GroupID 20

In this system, our customers are expected to be served at two points, one is front-desk operator who records the personal details of the caller and the nature of the caller's request and the other is expert operator who tries to help the caller with his/her problem.
* The inter-arrival times of calls are exponentially distributed with mean INTERARRIVAL_MEAN min.
* Incoming calls are first processed by the unskilled front-desk operator who records the personal details of the caller and the nature of the caller's request. When the operator is busy the customers are put on hold (they wait in a FCFS queue).
* The time it takes to collect and record the details of a caller is assumed to be LogNormal distributed with mean SERVICE_ONE_LOGNORMAL_MEAN and standard deviation of SERVICE_ONE_LOGNORMAL_STD minutes.
* Once this process is completed, the caller is directed to the expert operator, who tries to help the caller with his/her problem. As in the previous case, if the expert is busy the customers are put on hold (they wait in a FCFS queue).
* Every customer that joins the queue leaves the queue without getting service following an independent exponentially distributed time with mean PATIENCE_EXPONENTIAL_MEAN minutes. (reneging).
* The service time of the expert is Exponentially distributed with mean SERVICE_TWO_EXPONENTIAL_MEAN minutes.
* The expert operator takes 3-min breaks randomly through out the day. When the operator decides to take a break, he/she waits until completing all the customers already waiting for her/him. If new customers arrive during operators break, they wait in the FCFS queue until the operator serves them. The operator resumes service after the break.
* The number of breaks the expert operator wishes to take during an 8-hour shift is known to be distributed according to a Poisson distribution with a mean of BREAK_POISSON_MEAN_PERSHIFT breaks per shift.




In [1]:
import simpy
import random
import math

TOTALCALL stands for number of answered calls, run them SEPARATELY to obtain both simulations.

In [2]:
TOTALCALL = 1000

In [3]:
TOTALCALL = 5000

Global variables that can be changable for different simulation values, these values are given by the description page, except RANDOM_SEED.

In [4]:
RANDOM_SEED = 978  # To obtain the same results for random values

INTERARRIVAL_MEAN = 14.3    # Inter-arrival mean
SERVICE_ONE_LOGNORMAL_MEAN = 7.2   # Front-desk operation mean for Lognormal
SERVICE_ONE_LOGNORMAL_STD = 2.7   # Front-desk operation standard deviation for Lognormal
PATIENCE_EXPONENTIAL_MEAN = 60   # Customer reneging mean
SERVICE_TWO_EXPONENTIAL_MEAN = 10.2   # Expert operation mean
BREAK_POISSON_MEAN_PERSHIFT = 8   # Mean of break count per shift

Global set of variables for bookkeeping. Expected system responses are:
* Utilization of the front-desk operator
* Utilization of the expert operator
* Average Wotal Waiting Time
* Maximum Total Waiting Time to Total System Time Ratio
* Average number of people waiting to be served by the expert operator

Below array indexes are in the order of leaving calls not entering from the whole system.

In [5]:
LAST_MINUTE_ON_SYSTEM = 0  # Total Time 
TOTAL_WORKING_ONE = []     # Front-desk serve times for all calls
TOTAL_WORKING_TWO = []     # Expert serve times for all calls
TOTAL_WAITING_ONE = []     # Front-desk queue waiting times for all calls
TOTAL_WAITING_TWO = []     # Expert queue waiting times for all calls
TOTAL_WASTE_RATIO = []     # Idle ratio for all calls
LEFT_CUSTOMERS = 0         # Counter for leaving calls, termination will be determined by this variable
ISCONTINUING = True        # Kepps the boolean for system is active or not


## Call customer class

Call object will be added to the Environment after it is created in its own constructor.
arrivalTime is the arrival of the call by taking now as a initial point.
env is the simulation system. 
callNumber is the number of the Call class initialized. 
firstQueue is the front-desk queue, and secondQueue is the expert queue.

Important note about this simulation are:
* System will generate calls until TOTALCALL, specified, calls are left. Therefore, more than TOTALCALL calls may be generated but only first TOTALCALL left calls will be recorded.
* Reneging calls will counted as left customers.
* System stops after TOTALCALL calls are left, because it waits for remaining calls to finish their tasks. Fortunately, their records will not be taken into the consideration.


In [6]:
class Call(object):
    def __init__(self, arrivalTime, env,callNumber, firstQueue, secondQueue):
        self.env = env
        self.arrivalTime = arrivalTime
        self.frontDeskQueue = firstQueue
        self.expertDeskQueue = secondQueue
        env.process(self.run())
        self.callNumber = callNumber

    def run(self):
        global LEFT_CUSTOMERS, TOTAL_WORKING_ONE, TOTAL_WAITING_TWO, TOTAL_WAITING_ONE, TOTAL_WAITING_TWO, ISCONTINUING, LAST_MINUTE_ON_SYSTEM
        waiting_queue_one, waiting_queue_two, serve_time_one, serve_time_two = 0, 0, 0, 0
        
        yield self.env.timeout(self.arrivalTime)   # waits for interarrival time
        print("Arrived Call #" + str(self.callNumber) + " , TIME :" + str(self.env.now))
        
        if ISCONTINUING:    # If system is not finished continues creating new calls and assigns interarrival times.
            Call(arrivalCalc(), self.env, self.callNumber + 1, self.frontDeskQueue, self.expertDeskQueue)
        
        working_one = frontDeskCalc()    # Front-desk service time
        init_queue_one = self.env.now    # Initial time for queue
        with self.frontDeskQueue.request() as request: # Front-desk service queue system
            yield request
            print("Call #" + str(self.callNumber) + " entering front-desk, service time :" + str(working_one) + " , TIME :" + str(self.env.now))
            waiting_queue_one = self.env.now - init_queue_one  # Waiting time in queue one
            yield self.env.timeout(working_one)       # Serve time for this call
            print("Call #" + str(self.callNumber) + " leaving front-desk, TIME :" + str(self.env.now))
        
        serve_time_one = working_one    # Serve time is kept
        
        init_queue_two = self.env.now    # Expert service initial time
        with self.expertDeskQueue.request() as request:  # Expert service queue system
            patienced = patienceTime()  # Patience time for reneging
            outOfPatience = self.env.timeout(patienced)   # Reneging process
            yield request | outOfPatience   # Either call will get its request or it will be reneging
            waiting_queue_two = self.env.now - init_queue_two  # Waiting time in queue two
            if not request.triggered:    # If call is reneged than this will be true
                print("Call #" + str(self.callNumber) + " reneged, patience time was :" + str(patienced) + " , TIME :" + str(self.env.now))
                serve_time_two = 0    # Reneging call has 0 serve time
            else:                      # If call is served at time 
                service = serviceTime()
                print("Call #" + str(self.callNumber) + " entering expert service, service time :" + str(service) + " , TIME :" + str(self.env.now))
                yield self.env.timeout(service)  # Service for expert
                print("Call #" + str(self.callNumber) + " leaving expert service, TIME :" + str(self.env.now))
                serve_time_two = service

        LEFT_CUSTOMERS = LEFT_CUSTOMERS + 1   # Increments number of leaving calls
        
        if(LEFT_CUSTOMERS <= TOTALCALL):          # Only record first TOTALCALL calls
            TOTAL_WORKING_ONE.append(serve_time_one)
            TOTAL_WORKING_TWO.append(serve_time_two)
            TOTAL_WAITING_ONE.append(waiting_queue_one)
            TOTAL_WAITING_TWO.append(waiting_queue_two)
        
        if(LEFT_CUSTOMERS == TOTALCALL):    # If last call is leaving then stop generating new customert
            ISCONTINUING = False
            LAST_MINUTE_ON_SYSTEM = self.env.now

## Expert Class

This class is used for achieving break operation. It continues until system is done. Break operation is done by putting itself on second queue. So that we achieve goal of waiting for already requested calls and then giving a break. Otherwise expert may suffer from waiting whole queue to be empty to give a break.

In [7]:
class Expert(object):
    def __init__(self, env, secondQueue):
        self.env = env
        self.expertDeskQueue = secondQueue
        env.process(self.run())

    def run(self):
        while ISCONTINUING:   # Run until system is done.
            break_time = breakTime()   # Calculate break request time
            print("Expert wanna break after : " + str(break_time) + " min, TIME :" + str(self.env.now))
            yield self.env.timeout(break_time)   # Wait until break request
            print("Expert request its break on service 2, TIME :" + str(self.env.now))
            with self.expertDeskQueue.request() as request:  # Request for queue two
                yield request
                if not ISCONTINUING:   
                    return
                print("Expert entering break, TIME :" + str(self.env.now))
                yield self.env.timeout(3)  # Expert waits 3 minute in break
                print("Expert leaving break, TIME :" + str(self.env.now))

### Random function generators


In [8]:
def arrivalCalc():   # Generates interarrival time
    return random.expovariate(1/INTERARRIVAL_MEAN)   

# Given parameter for lognormal should be normal distribution so we convert it to normal distribution mean and std by using lognormal mean and std.
def frontDeskCalc():   # Generates front-desk service time
    n_mean = math.log(SERVICE_ONE_LOGNORMAL_MEAN/(math.sqrt(1 + ((SERVICE_ONE_LOGNORMAL_STD**2)/(SERVICE_ONE_LOGNORMAL_MEAN**2)))))
    n_std = math.sqrt(math.log(1 + ((SERVICE_ONE_LOGNORMAL_STD**2)/(SERVICE_ONE_LOGNORMAL_MEAN**2))))
    return random.lognormvariate(n_mean, n_std)

def patienceTime():  # Reneging time generator
    return random.expovariate(1/PATIENCE_EXPONENTIAL_MEAN)

def serviceTime():  # Generates expert service time
    return random.expovariate(1/SERVICE_TWO_EXPONENTIAL_MEAN)

def breakTime():   # Inter break times by using given poisson mean
    return random.expovariate(1/(8*60/BREAK_POISSON_MEAN_PERSHIFT))

### Main function

In [9]:
callCenter = simpy.Environment()
frontDeskQueue = simpy.Resource(callCenter, capacity=1)   # Front-desk queue
expertDeskQueue = simpy.Resource(callCenter, capacity=1)   # Expert queue

random.seed(RANDOM_SEED)   # Random seed is set
Call(arrivalCalc(), callCenter, 1, frontDeskQueue, expertDeskQueue)  # First call is created
Expert(callCenter, expertDeskQueue)   # Expert is created 

callCenter.run()   

for i in range(TOTALCALL):   # Calculates each individual waster ratio of each call
    TOTAL_WASTE_RATIO.append((TOTAL_WAITING_ONE[i] + TOTAL_WAITING_TWO[i])/(TOTAL_WAITING_ONE[i] + TOTAL_WAITING_TWO[i] + TOTAL_WORKING_ONE[i] + TOTAL_WORKING_TWO[i]))

print("UTILIZATION OF FRONT DESK OPERATOR IS " + str(100*sum(TOTAL_WORKING_ONE)/LAST_MINUTE_ON_SYSTEM) + "%")
print("UTILIZATION OF FRONT EXPERT OPERATOR IS " + str(100*sum(TOTAL_WORKING_TWO)/LAST_MINUTE_ON_SYSTEM) + "%")
print("AVERAGE TOTAL WAITING TIME IS " + str((sum(TOTAL_WAITING_ONE) + sum(TOTAL_WAITING_TWO))/TOTALCALL))
print("MAX WASTE RATIO " + str(max(TOTAL_WASTE_RATIO)))
print("AVERAGE NUMBER ON QUEUE TWO " + str((sum(TOTAL_WAITING_TWO)/TOTALCALL) * (TOTALCALL/LAST_MINUTE_ON_SYSTEM)))
print("SYSTEM TIME IS : " + str(LAST_MINUTE_ON_SYSTEM))

Expert wanna break after : 10.104408034789104 min, TIME :0
Expert request its break on service 2, TIME :10.104408034789104
Expert entering break, TIME :10.104408034789104
Expert leaving break, TIME :13.104408034789104
Expert wanna break after : 5.855815390564933 min, TIME :13.104408034789104
Expert request its break on service 2, TIME :18.960223425354037
Expert entering break, TIME :18.960223425354037
Expert leaving break, TIME :21.960223425354037
Expert wanna break after : 97.24780396433752 min, TIME :21.960223425354037
Arrived Call #1 , TIME :63.19079719550898
Call #1 entering front-desk, service time :5.473090236176437 , TIME :63.19079719550898
Call #1 leaving front-desk, TIME :68.66388743168541
Call #1 entering expert service, service time :9.583151533461166 , TIME :68.66388743168541
Arrived Call #2 , TIME :74.087257139436
Call #2 entering front-desk, service time :5.350389364248514 , TIME :74.087257139436
Call #1 leaving expert service, TIME :78.24703896514657
Call #2 leaving fron

Call #249 leaving expert service, TIME :3806.0965864146374
Call #250 entering expert service, service time :5.695671369501838 , TIME :3806.0965864146374
Arrived Call #253 , TIME :3810.666800711376
Call #250 leaving expert service, TIME :3811.792257784139
Call #251 entering expert service, service time :21.075189733543695 , TIME :3811.792257784139
Call #252 leaving front-desk, TIME :3814.301952262752
Call #253 entering front-desk, service time :8.948388361621491 , TIME :3814.301952262752
Arrived Call #254 , TIME :3821.4932598057444
Call #253 leaving front-desk, TIME :3823.2503406243736
Call #254 entering front-desk, service time :10.704353316593707 , TIME :3823.2503406243736
Arrived Call #255 , TIME :3827.5656125704386
Call #251 leaving expert service, TIME :3832.8674475176826
Call #252 entering expert service, service time :0.19597784568302082 , TIME :3832.8674475176826
Call #252 leaving expert service, TIME :3833.0634253633657
Call #253 entering expert service, service time :2.1018478

Call #523 leaving front-desk, TIME :7476.273173802946
Call #524 entering front-desk, service time :5.698671607681388 , TIME :7476.273173802946
Call #523 entering expert service, service time :39.12906387639989 , TIME :7476.273173802946
Expert request its break on service 2, TIME :7477.003674051832
Call #524 leaving front-desk, TIME :7481.971845410628
Arrived Call #525 , TIME :7482.293711496575
Call #525 entering front-desk, service time :7.189657100985243 , TIME :7482.293711496575
Call #525 leaving front-desk, TIME :7489.483368597561
Call #525 reneged, patience time was :8.52035652070201 , TIME :7498.003725118263
Call #524 reneged, patience time was :18.64130539060166 , TIME :7500.6131508012295
Call #523 leaving expert service, TIME :7515.402237679346
Expert entering break, TIME :7515.402237679346
Expert leaving break, TIME :7518.402237679346
Expert wanna break after : 42.592024404505715 min, TIME :7518.402237679346
Arrived Call #526 , TIME :7518.685736635406
Call #526 entering front-d

Call #797 leaving front-desk, TIME :11301.461920890455
Call #794 leaving expert service, TIME :11302.575407278375
Call #795 entering expert service, service time :4.711923505473448 , TIME :11302.575407278375
Call #797 reneged, patience time was :3.804563656014421 , TIME :11305.26648454647
Call #795 leaving expert service, TIME :11307.287330783849
Call #796 entering expert service, service time :14.58812558926879 , TIME :11307.287330783849
Arrived Call #798 , TIME :11309.38724877556
Call #798 entering front-desk, service time :7.058860172089458 , TIME :11309.38724877556
Arrived Call #799 , TIME :11315.208162646082
Call #798 leaving front-desk, TIME :11316.44610894765
Call #799 entering front-desk, service time :8.029551648427422 , TIME :11316.44610894765
Arrived Call #800 , TIME :11318.876362836876
Expert request its break on service 2, TIME :11319.352800139297
Call #796 leaving expert service, TIME :11321.875456373118
Call #798 entering expert service, service time :1.42989040865401 , 

Call #1061 leaving expert service, TIME :14987.751018164701
Call #1062 leaving front-desk, TIME :14993.68726303782
Call #1062 entering expert service, service time :3.9377284399816355 , TIME :14993.68726303782
Arrived Call #1063 , TIME :14995.905700276035
Call #1063 entering front-desk, service time :7.965811201743064 , TIME :14995.905700276035
Call #1062 leaving expert service, TIME :14997.624991477802
Arrived Call #1064 , TIME :15002.610060920453
Arrived Call #1065 , TIME :15003.309446098725
Call #1063 leaving front-desk, TIME :15003.871511477777
Call #1064 entering front-desk, service time :9.741778072671499 , TIME :15003.871511477777
Call #1063 entering expert service, service time :1.113283465924671 , TIME :15003.871511477777
Call #1063 leaving expert service, TIME :15004.9847949437
Call #1064 leaving front-desk, TIME :15013.613289550449
Call #1065 entering front-desk, service time :12.879646033598238 , TIME :15013.613289550449
Call #1064 entering expert service, service time :3.7

...

Call #3879 entering front-desk, service time :7.54067151913825 , TIME :55370.855931604616
Call #3878 entering expert service, service time :3.136490426428261 , TIME :55370.855931604616
Call #3878 leaving expert service, TIME :55373.99242203104
Expert request its break on service 2, TIME :55375.179536000986
Expert entering break, TIME :55375.179536000986
Expert leaving break, TIME :55378.179536000986
Expert wanna break after : 15.488040668647512 min, TIME :55378.179536000986
Call #3879 leaving front-desk, TIME :55378.39660312375
Call #3879 entering expert service, service time :2.704085910133111 , TIME :55378.39660312375
Call #3879 leaving expert service, TIME :55381.10068903388
Arrived Call #3880 , TIME :55392.92345028522
Call #3880 entering front-desk, service time :7.21951891452818 , TIME :55392.92345028522
Expert request its break on service 2, TIME :55393.667576669635
Expert entering break, TIME :55393.667576669635
Expert leaving break, TIME :55396.667576669635
Expert wanna break a

Call #4130 leaving expert service, TIME :58555.754718545744
Call #4131 leaving front-desk, TIME :58556.35315684348
Call #4132 entering front-desk, service time :10.11968322389744 , TIME :58556.35315684348
Call #4131 entering expert service, service time :4.272687110544246 , TIME :58556.35315684348
Arrived Call #4133 , TIME :58560.29833043606
Call #4131 leaving expert service, TIME :58560.625843954025
Call #4132 leaving front-desk, TIME :58566.47284006738
Call #4133 entering front-desk, service time :9.514715858539647 , TIME :58566.47284006738
Call #4132 entering expert service, service time :5.410209638190635 , TIME :58566.47284006738
Call #4132 leaving expert service, TIME :58571.88304970557
Call #4133 leaving front-desk, TIME :58575.987555925916
Call #4133 entering expert service, service time :5.277828197359964 , TIME :58575.987555925916
Call #4133 leaving expert service, TIME :58581.265384123275
Arrived Call #4134 , TIME :58593.645063916716
Call #4134 entering front-desk, service t

Expert entering break, TIME :61870.2871159853
Expert leaving break, TIME :61873.2871159853
Expert wanna break after : 7.722785809725467 min, TIME :61873.2871159853
Call #4361 leaving front-desk, TIME :61874.788188331324
Call #4361 entering expert service, service time :8.763377893850356 , TIME :61874.788188331324
Expert request its break on service 2, TIME :61881.00990179503
Call #4361 leaving expert service, TIME :61883.55156622517
Expert entering break, TIME :61883.55156622517
Expert leaving break, TIME :61886.55156622517
Expert wanna break after : 268.99812786103377 min, TIME :61886.55156622517
Arrived Call #4362 , TIME :61926.48520948525
Call #4362 entering front-desk, service time :7.264497140565467 , TIME :61926.48520948525
Call #4362 leaving front-desk, TIME :61933.749706625815
Call #4362 entering expert service, service time :5.062824323510554 , TIME :61933.749706625815
Call #4362 leaving expert service, TIME :61938.812530949326
Arrived Call #4363 , TIME :61940.47997486773
Call

Expert request its break on service 2, TIME :64855.28170981611
Arrived Call #4564 , TIME :64861.10351626863
Call #4564 entering front-desk, service time :6.759143136190079 , TIME :64861.10351626863
Call #4562 leaving expert service, TIME :64863.250971426605
Call #4563 entering expert service, service time :18.084494858362007 , TIME :64863.250971426605
Call #4564 leaving front-desk, TIME :64867.86265940482
Arrived Call #4565 , TIME :64869.8202512357
Call #4565 entering front-desk, service time :7.134220683894192 , TIME :64869.8202512357
Arrived Call #4566 , TIME :64874.362580559675
Call #4565 leaving front-desk, TIME :64876.9544719196
Call #4566 entering front-desk, service time :14.756734243637263 , TIME :64876.9544719196
Call #4563 leaving expert service, TIME :64881.33546628497
Expert entering break, TIME :64881.33546628497
Expert leaving break, TIME :64884.33546628497
Expert wanna break after : 22.07590871292455 min, TIME :64884.33546628497
Call #4564 entering expert service, servic

Call #4793 leaving front-desk, TIME :68302.19029022302
Call #4794 entering front-desk, service time :9.360827188552086 , TIME :68302.19029022302
Call #4792 leaving expert service, TIME :68305.31619184339
Call #4793 entering expert service, service time :13.073925961837952 , TIME :68305.31619184339
Expert request its break on service 2, TIME :68311.01027188549
Call #4794 leaving front-desk, TIME :68311.55111741157
Call #4793 leaving expert service, TIME :68318.39011780523
Expert entering break, TIME :68318.39011780523
Expert leaving break, TIME :68321.39011780523
Expert wanna break after : 34.05905726219838 min, TIME :68321.39011780523
Call #4794 entering expert service, service time :8.39693699505097 , TIME :68321.39011780523
Arrived Call #4795 , TIME :68324.07604301066
Call #4795 entering front-desk, service time :4.439358101340093 , TIME :68324.07604301066
Call #4795 leaving front-desk, TIME :68328.515401112
Call #4794 leaving expert service, TIME :68329.78705480028
Call #4795 enteri